In [ ]:
import numpy as np
import talib 
import websocket
import json
import matplotlib.pyplot as plt
import datetime

In [ ]:
# defining TRADE parameters
amount = 10000                               # money in the trading account
portfolio = 0                                # no. of stocks 
investments = []                             # track for investments
money_end = amount                           # left over balance in trading account

trade_fund = money_end *0.05

long_position = 0
short_position = 0

# defining DATA EXTRACTION parameters (using websocket from Binance)
symbol = 'btcusdt'                           # asset symbol
interval = '1m'                              # asset interval
socket = f'wss://stream.binance.com:9443/ws/{symbol}@kline_{interval}'

# defining CANDLESTICK DATA VISUALITION parameters
opens=[]
closes=[]
highs=[]
lows=[]
candle_close_time=[]

# defining indicator parameter
SMA_timeperiod = 9
aroon_time_period = 3
aroon_up = []
aroon_down = []
# Aroon_high = np.array(highs)
# Aroon_low = np.array(lows)


In [ ]:
# Buy and Sell Functions

def buy (allocated_amount,price):
    global portfolio,investments,money_end
    
    if money_end > 0:
        quantity = allocated_amount/price
        portfolio += quantity
        money_end -= quantity*price

        print(f'Quantity Bought: {quantity}')
        print(f'Updated Portfolio: {portfolio}')
        print(f'Remaning Funds: {money_end}')
        
        if  investments == []:                           # To update investment history
            investments.append(allocated_amount)
            print(f'Investment History:{investments}')
        else:
            investments.append(allocated_amount)
            investments[-1] += investments[-2]
            print(f'Investment History:{investments}')
    
    else:
        print("Not Enough balance in the account to execute the trade!")
        # print(f'Final Quantity Bought: {quantity}')
        # print(f'Final Updated Portfolio: {portfolio}')
        # print(f'Final Remaning Funds: {money_end}')



def sell (allocated_amount,price):
    global portfolio,investments,money_end
    
    if money_end > 0:
        quantity = allocated_amount/price
        portfolio -= quantity
        money_end += quantity*price

        print(f'Quantity Sold: {quantity}')
        print(f'Updated Portfolio: {portfolio}')
        print(f'Total Funds: {money_end}')
        
        if  investments == []:
            investments.append(-allocated_amount)
            print(f'Investment History:{investments}')
        else:
            investments.append(-allocated_amount)
            investments[-1] += investments[-2]
            print(f'Investment History:{investments}')
    
    else:
        print("Not Enough balance in the account to execute the trade!")
        # print(f'Final Quantity Bought: {quantity}')
        # print(f'Final Updated Portfolio: {portfolio}')
        # print(f'Final Remaning Funds: {money_end}')


In [ ]:
# EXTRACTING DATA

def on_close(ws):
    print("connection closed")


def on_message(ws, message):
    global opens,closes,highs,lows, candle_close_time, aroon_up, aroon_down
    json_message = json.loads(message)
    candlesticks = json_message["k"]
    candlesticks_closed, candlesticks_closedTime, open, close, high, low = candlesticks['x'], candlesticks['T'], candlesticks['o'], candlesticks['c'], candlesticks['h'], candlesticks['l']
    


    if candlesticks_closed:               # All the below lists will appended as soon as candle is closed
        opens.append(float(open))
        highs.append(float(high))
        closes.append(float(close))
        lows.append(float(low))
        candle_close_time.append(float(candlesticks_closedTime))

        aroon_up, aroon_down = talib.AROON(np.array(highs),np.array(lows),aroon_time_period) # calculating aroon data using talib lib
        SMA_9 = talib.SMA(np.array(closes), SMA_timeperiod)                                  # calculating SMA data using talib lib
        # print(aroon_up,aroon_down)
        # print(SMA_9)
        
        
        utc_datetimes = []

        for timestamp in candle_close_time:                             #loop to convert unix timestamp to UTC time
            timestamp /= 1000
            utc_datetime = datetime.datetime.utcfromtimestamp(timestamp)
            utc_datetimes.append(utc_datetime.strftime('%Y-%m-%d %H:%M:%S'))

    # BUY STRATEGY
        if  long_position == 0 and aroon_up[-1] > aroon_down[-1] and aroon_up[-2] < aroon_down[-2] and \
            closes[-1] >  SMA_9[-1]:                                                                          # Buy when there is cross over between aroonup and aroondown with closing price above 9 SMA
            print(f'Buy order initiated at price ${closes[-1]}')
            print(f'Timestamp:{candle_close_time[-1]}')
            long_position = 1
            print(f'In strategy buy flag :{long_position}')
            buy (trade_fund, price= closes[-1])
            print("____________________________________________")
            buy_quantity = trade_fund / closes[-1]
            # buy_price = closes[-1]
            

        if long_position == 1 and closes[-1] < SMA_9[-1] :                                                 # Exit trade when candle closes below 9 SMA 
            buytrade_exit_price = closes[-1]
            print(f'Current price is : ${closes[-1]}. Waiting for price to reach {buytrade_exit_price}')
            print(f'Buy exit initiated at price ${closes[-1]}')
            print(f'Timestamp:{candle_close_time[-1]}')
            long_position = 0
            print(f'In strategy buy flag :{long_position}')
            afterbuy_trade_fund = buy_quantity * buytrade_exit_price
            sell(afterbuy_trade_fund, price=buytrade_exit_price)
            print("****************END OF TRADE****************")
            print(" ")

    # SELL STRATEGY
        if  short_position == 0 and aroon_down[-1] > aroon_up[-1] and aroon_down[-2] < aroon_up[-2] and \
            closes[-1] <  SMA_9[-1]:                                                                         # Sell when there is cross over between aroondown and aroonup with closing price below 9 SMA
            print(f'Short order initiated at price ${closes[-1]}')
            print(f'Timestamp:{candle_close_time[-1]}')
            short_position = 1
            print(f'In strategy sell flag :{short_position}')
            sell (trade_fund, price= closes[-1])
            print("____________________________________________")
            sell_quantity = trade_fund / closes[-1]
            # sell_price = closes[-1]
            

        if  short_position == 1 and closes[-1] > SMA_9[-1]:                                                # Exit trade when candle closes above 9 SMA
            selltrade_exit_price = closes[-1]
            print(f'Current price is : ${closes[-1]}. Waiting for price to reach {selltrade_exit_price}')
            print(f'Short exit initiated at price ${selltrade_exit_price}')
            print(f'Timestamp:{candle_close_time[-1]}')
            short_position = 0
            print(f'In strategy buy flag :{short_position}')
            aftersell_trade_fund = sell_quantity * selltrade_exit_price
            buy(aftersell_trade_fund, price=selltrade_exit_price)
            print("****************END OF TRADE****************")
            print(" ")

ws = websocket.WebSocketApp(socket, on_message = on_message, on_close = on_close)
ws.run_forever()
